# Setup

In [1]:
import numpy as np
import pandas as pd

Read in earthquake.csv

In [2]:
earthquakes = pd.read_csv('../ch_04/exercises/earthquakes.csv')
faang = pd.read_csv('../ch_04/exercises/faang.csv', index_col='date', parse_dates=True)


# Task
1. Select eathquake in Japan with magnitude 4.9 or greater and type mb

In [ ]:
earthquakes.query("magType == 'mb' and parsed_place == 'Japan' and mag >= 4.9")[['mag', 'magType', 'place', 'parsed_place']]

2. Create bins (0,1], (1,2] etc for magnitude type ml and count how many are in each bin.

In [ ]:
earthquakes.query("magType == 'ml'").assign(
    mag_bin=lambda x: pd.cut(x.mag, np.arange(0, 10))
).mag_bin.value_counts().sort_index()

3. Using faang.csv group by ticker and resample to monthly frequency.

In [ ]:
faang.groupby('ticker').resample('1M').agg(
    {
    'open' : np.mean,
    'high' : np.max,
    'low' : np.min,
    'close' : np.mean,
    'volume' : np.sum
    }
)

4. Build a crosstab with earthquake data between *tsunami* and *magType* column, show max magnitude per column

In [ ]:
pd.crosstab(earthquakes.tsunami, earthquakes.magType, values=earthquakes.mag, aggfunc='max')

5. Calculate rolling 60-day aggregations of the OHLC data by ticker for FAANG data.

In [ ]:
faang.groupby('ticker').rolling('60D').agg(
    {
    'open' : np.mean,
    'high' : np.max,
    'low' : np.min,
    'close' : np.mean,
    'volume' : np.sum
    }
)

6. Create a pivot table with the FAANG data.

In [ ]:
faang.pivot_table(index='ticker', aggfunc='mean')

7. Calculate Z-scores for Amazon data in Q4 of 2018 using *apply( )*

In [ ]:
faang_Q4 = faang.loc['2018-Q4'] # Get 2018 Q4 data
faang_Q4.query("ticker == 'AMZN'").drop(columns='ticker').apply(lambda x: x.sub(x.mean()).div(x.std()))

8. Add event descriptions

- Create dataframe with appropriate ticker, dates and event descriptions

In [ ]:
faang_event = pd.DataFrame(
    {
        'ticker' : 'FB',
        'date' : pd.to_datetime(['2018-07-25', '2018-03-19', '2018-03-20']),
        'event' : ['Disapointing user growth announced after close.', 'Cambridge Analytics story', 'FTC investigation']
    }
).set_index(['date', 'ticker'])


- Merge with FAANG date using outer join

In [ ]:
events = faang.reset_index().set_index(['date', 'ticker']).join(faang_event, how='outer')

- Check it worked

In [ ]:
events.query("ticker =='FB' and date > '2018-03-18' and date < '2018-07-26'")

9. Use *transform( )* method

In [ ]:
faang = faang.reset_index().set_index(['ticker', 'date'])
faang_index = (faang / faang.groupby(level='ticker').transform('first'))

faang_index.groupby(level='ticker').head()

10. European Centre for Disease Prevention and Control (ECDC) snapshot of data through September 18 2020

* Part 1
    - Read the data
    - Tidy up dateRep to datetime format and set as index
    - Replace United_States_of_America with USA
    - Replace United_Kingdom with UK

In [2]:
covid19 = pd.read_csv('../ch_04/exercises/covid19_cases.csv')\
    .assign(date=lambda x: pd.to_datetime(x.dateRep, format='%d/%m/%Y'))\
    .set_index('date')\
    .replace('United_States_of_America', 'USA')\
    .replace('United_Kingdom', 'UK')\
    .sort_index()
covid19.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
date,,,,,,,,,,,,
2019-12-31,31/12/2019,31,12,2019,0,0,Belgium,BE,BEL,11455519.0,Europe,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Mexico,MX,MEX,127575529.0,America,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Ecuador,EC,ECU,17373657.0,America,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Russia,RU,RUS,145872260.0,Europe,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Netherlands,NL,NLD,17282163.0,Europe,NaN


* Part 2
    - Create an index of top 5 countries based on cases
    - Get the index of the date where max cases for each country

In [10]:
top_5_countries = covid19.groupby('countriesAndTerritories').cases.sum().nlargest(5).index
covid19[covid19.countriesAndTerritories.isin(top_5_countries)]\
    .groupby('countriesAndTerritories').cases.idxmax()

countriesAndTerritories
Brazil   2020-07-30
India    2020-09-17
Peru     2020-08-17
Russia   2020-07-18
USA      2020-07-25
Name: cases, dtype: datetime64[ns]

* Part 3
    - Find the 7-day average change in COVID-19 cases for the last week in the data for the countries found in part 2

In [18]:
covid19.groupby(['countriesAndTerritories', pd.Grouper(freq='1D')]).cases.sum()\
    .unstack(0).diff().rolling(7).mean().last('1W')[top_5_countries]



countriesAndTerritories,USA,India,Brazil,Russia,Peru
date,,,,,
2020-09-14,473.714286,181.285714,35.285714,36.285714,73.142857
2020-09-15,1513.000000,1142.857143,697.428571,46.285714,377.571429
2020-09-16,3478.714286,59.571429,3196.285714,61.428571,-65.000000
2020-09-17,-1047.000000,308.428571,143.428571,810.000000,-29.428571
2020-09-18,865.714286,-18.142857,-607.714286,-688.428571,-227.571429
2020-09-19,306.857143,-604.714286,-560.142857,57.285714,-41.285714


* Part 4
    - Find the first date that each country, other than China, had cases

In [31]:
covid19.pivot(columns='countriesAndTerritories', values='cases')\
    .drop(columns='China')\
    .apply(lambda x: x[x > 0].index.min())\
    .sort_values()\
    .rename(lambda x: x.replace('_', ' '))

countriesAndTerritories
Thailand         2020-01-13
Japan            2020-01-15
South Korea      2020-01-20
Taiwan           2020-01-21
USA              2020-01-21
                    ...    
Yemen            2020-04-10
Western Sahara   2020-04-26
Tajikistan       2020-05-01
Comoros          2020-05-02
Lesotho          2020-05-15
Length: 209, dtype: datetime64[ns]